In [1]:
import pandas as pd
import pickle

In [ ]:
data = pd.read_csv("data/train.csv")

## Open InferSent embeddings and create dictionaries

In [3]:
pickle_in = open("data/emb_s_25k.pickle", "rb")
sen1 = pickle.load(pickle_in)
pickle_in.close()

In [4]:
pickle_in = open("data/emb_s_50k.pickle", "rb")
sen2 = pickle.load(pickle_in)
pickle_in.close()

In [5]:
pickle_in = open("data/emb_s_75k.pickle", "rb")
sen3 = pickle.load(pickle_in)
pickle_in.close()

In [6]:
pickle_in = open("data/emb_s_rest.pickle", "rb")
sen4 = pickle.load(pickle_in)
pickle_in.close()

In [7]:
dict_sen = dict(sen1)
del sen1

In [8]:
dict_sen.update(sen2)
del sen2

In [9]:
dict_sen.update(sen3)
del sen3

In [10]:
dict_sen.update(sen4)
del sen4

In [11]:
pickle_in = open("data/emb_q_25k.pickle", "rb")
que1 = pickle.load(pickle_in)
pickle_in.close()

In [12]:
pickle_in = open("data/emb_q_50k.pickle", "rb")
que2 = pickle.load(pickle_in)
pickle_in.close()

In [13]:
pickle_in = open("data/emb_q_75k.pickle", "rb")
que3 = pickle.load(pickle_in)
pickle_in.close()

In [14]:
pickle_in = open("data/emb_q_rest.pickle", "rb")
que4 = pickle.load(pickle_in)
pickle_in.close()

In [15]:
dict_que = dict(que1)
del que1

In [16]:
dict_que.update(que2)
del que2

In [17]:
dict_que.update(que3)
del que3

In [18]:
dict_que.update(que4)
del que4

## Add splitted contexts, answers and embeddings to the Data Frame

In [19]:
from textblob import TextBlob
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\czajk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
def split_context(context):
    blob = TextBlob(context)
    sentences = [item.raw for item in blob.sentences]
    return sentences

In [21]:
def find_answer(qa):
    for i in range(len(qa["sentences"])):
        if not isinstance(qa["text"], str):
            qa["text"] = str(qa["text"])
        if qa["text"] in qa["sentences"][i]:
            return i
    return -1

In [22]:
data["sentences"] = data["context"].apply(split_context)
data["ans_sentence"] = data.apply(find_answer, axis = 1)

In [23]:
def get_que_emb(question):
    if question in dict_que:
        return dict_que[question]
    return np.zeros(4096)

In [24]:
import numpy as np

def get_sent_emb(sentences):
    res = [dict_sen[item][0] if item in dict_sen else np.zeros(4096) for item in sentences]
    return res

In [25]:
data["que_emb"] = data["question"].apply(get_que_emb)
data["sent_emb"] = data["sentences"].apply(get_sent_emb)

In [26]:
del dict_que
del dict_sen

In [27]:
del data["context"]

In [28]:
data.head(15)

,question,answer_start,text,sentences,ans_sentence,que_emb,sent_emb
0,When did Beyonce start becoming popular?,269,in the late 1990s,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.11226777, 0.054507274, 0.107090585, 0.0954...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
1,What areas did Beyonce compete in when she was...,207,singing and dancing,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.12392643, 0.08186243, 0.05142356, -0.01727...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
2,When did Beyonce leave Destiny's Child and bec...,526,2003,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,3,"[[0.04034418, 0.12923701, 0.03330859, 0.022752...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
3,In what city and state did Beyonce grow up?,166,"Houston, Texas",[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.082195975, 0.0778922, 0.016079443, 0.04441...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
4,In which decade did Beyonce become famous?,276,late 1990s,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.10957716, 0.020397408, 0.061727244, 0.0263...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
5,In what R&B group was she the lead singer?,320,Destiny's Child,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.04265439, 0.11198192, 0.10183123, -0.00464...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
6,What album made her a worldwide known artist?,505,Dangerously in Love,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,3,"[[0.104184076, 0.06301832, 0.12119591, -0.0010...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
7,Who managed the Destiny's Child group?,360,Mathew Knowles,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,2,"[[0.06858502, 0.11684115, -0.030445112, -0.027...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
8,When did Beyoncé rise to fame?,276,late 1990s,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.09504555, 0.01654029, 0.10866133, 0.017700...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."
9,What role did Beyoncé have in Destiny's Child?,290,lead singer,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.084585585, 0.12407758, 0.029805489, -0.005...","[[0.06352395, 0.17397285, 0.10705582, 0.073170..."


## Count Euclidean and cosine distances between the question and each answer

In [29]:
from scipy import spatial

def eucl_dis(record):
    res = [0] * len(record["sent_emb"])
    for i in range(len(record["sent_emb"])):
        res[i] = spatial.distance.euclidean(record["sent_emb"][i],record["que_emb"][0])
    return res

In [30]:
def cosine_dis(record):
    res = [0]*len(record["sent_emb"])
    for i in range(len(record["sent_emb"])):
        res[i] = spatial.distance.cosine(record["sent_emb"][i],record["que_emb"][0])
    return res

In [34]:
data["eucl_dis"] = data.apply(eucl_dis, axis = 1)

In [35]:
data["cosine_dis"] = data.apply(cosine_dis, axis = 1)

In [36]:
data.head(10)

,question,answer_start,text,sentences,ans_sentence,que_emb,sent_emb,eucl_dis,cosine_dis
0,When did Beyonce start becoming popular?,269,in the late 1990s,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.11226777, 0.054507274, 0.107090585, 0.0954...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.877357482910156, 4.7811198234558105, 3.9897...","[0.46874332427978516, 0.44890081882476807, 0.4..."
1,What areas did Beyonce compete in when she was...,207,singing and dancing,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.12392643, 0.08186243, 0.05142356, -0.01727...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.541980266571045, 4.213067531585693, 3.64840...","[0.38697999715805054, 0.3235163688659668, 0.36..."
2,When did Beyonce leave Destiny's Child and bec...,526,2003,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,3,"[[0.04034418, 0.12923701, 0.03330859, 0.022752...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.0321784019470215, 3.8517632484436035, 3.576...","[0.2884094715118408, 0.26048868894577026, 0.33..."
3,In what city and state did Beyonce grow up?,166,"Houston, Texas",[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.082195975, 0.0778922, 0.016079443, 0.04441...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.703168869018555, 4.480725288391113, 3.85302...","[0.42351508140563965, 0.3781578540802002, 0.41..."
4,In which decade did Beyonce become famous?,276,late 1990s,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.10957716, 0.020397408, 0.061727244, 0.0263...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.48006010055542, 4.477153301239014, 3.738474...","[0.37501227855682373, 0.3775840997695923, 0.38..."
5,In what R&B group was she the lead singer?,320,Destiny's Child,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.04265439, 0.11198192, 0.10183123, -0.00464...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.228331565856934, 3.993987560272217, 3.33958...","[0.3236287236213684, 0.2818150520324707, 0.299..."
6,What album made her a worldwide known artist?,505,Dangerously in Love,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,3,"[[0.104184076, 0.06301832, 0.12119591, -0.0010...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.465325832366943, 4.358132362365723, 3.45290...","[0.37130141258239746, 0.35230696201324463, 0.3..."
7,Who managed the Destiny's Child group?,360,Mathew Knowles,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,2,"[[0.06858502, 0.11684115, -0.030445112, -0.027...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.858935832977295, 4.607327461242676, 3.53121...","[0.4661130905151367, 0.4072984457015991, 0.352..."
8,When did Beyoncé rise to fame?,276,late 1990s,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.09504555, 0.01654029, 0.10866133, 0.017700...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.888204574584961, 4.848261833190918, 4.16055...","[0.47260910272598267, 0.46645498275756836, 0.5..."
9,What role did Beyoncé have in Destiny's Child?,290,lead singer,[Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ ...,1,"[[0.084585585, 0.12407758, 0.029805489, -0.005...","[[0.06352395, 0.17397285, 0.10705582, 0.073170...","[4.509896278381348, 4.38243293762207, 3.742840...","[0.38141047954559326, 0.3573759198188782, 0.38..."


In [38]:
data.to_csv("data/processed.csv", index=None)